# import library

In [ ]:
# these packages are maybe useful

import os
import sys
import time
import random
import argparse
import traceback

import numpy as np
import matplotlib.pyplot as plt

## GPU environment variables

In notebook, maybe use %env will be a better solution, but not a general one

In [ ]:
gpu = "1"

os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'          # for TensorFlow
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = gpu

# import code

%load filename

%%writefil

In [ ]:
%load_ext autoreload
%autoreload 2

from ExpUtils import *

# model arch

In [ ]:
# also can load from file

# train function

In [ ]:
def train(args):
    set_framework_seed(args.seed, args.debug)
    
    # train_set, test_set = train_l, train_ul, test_set = load_dataset(args.data_dir, valid=False, dataset_seed=args.seed)
    # wlog("size of dataset".format(train_set.size()))
    
    arch = getattr(CNN, args.arch)
    model = arch(args)
    
    if args.debug:
        # for pytorch, or weight initialization in the model
        # weights init is based on numpy, so only need np.random.seed()
        np.random.seed(args.seed)
        model.apply(weights_init_uniform)
        
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    # use linear lr decay or some other schedulers
    # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=args.lr_decay)
    start_epoch = 0
    if args.resume:
        # resume example
        checkpoint = load_checkpoint_by_marker(args, exp_marker)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch']
        for state in optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor):
                    state[k] = v.to(args.device)
    model = model.to(args.device)
    
    for epoch/iteration in range(total_steps):
        images, labels = train_set
        
        # loss = ...
        
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        if epoch/iteration % args.log_interval == 0:
            n_err, test_loss = evaluate_classifier(model, test_loader, args.device)
            acc = 1 - n_err / len(test_set)
            print("Epoch %d, test acc: %.5f" % (epoch, acc))